# Config enviroment

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"

# Imports

In [1]:
import torch
from torch import nn
from torch.utils.data import TensorDataset, DataLoader

import numpy as np

from util_func import read_EPCOR, preprocess_EPCOR

# Prepare data

# Models

# Train and Evaluate

# Main

In [ ]:
# Constants
data_root = "../Datasets/Time_Series_Datasets/EPCOR/"
train_prop = 0.8
lookback = 12
batch_size = 64
shuffle = True

# Read Dataset
df = read_EPCOR(data_root, force_reload=True)
(X_train, y_train), (X_test, y_test) = preprocess_EPCOR(df, train_prop, lookback)

# Create Datasets
train_dataset = TensorDataset(torch.from_numpy(X_train), torch.from_numpy(y_train))
test_dataset = TensorDataset(torch.from_numpy(X_test), torch.from_numpy(y_test))

# Create Dataloaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=shuffle, drop_last=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=shuffle, drop_last=True)

# Set device
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("==> Use accelerator: ", device)